# Module 2 Preprocessing

In [ ]:
import pyspark

In [ ]:
# Start a Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('abc').getOrCreate()


## Normalizer

In [ ]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import Vectors

In [ ]:
df4 = spark.createDataFrame([
(1,Vectors.dense([10.0,10000.0,1.0]),),
(2,Vectors.dense([20.0,20000.0,2.0]),),
(3,Vectors.dense([30.0,30000.0,3.0]),)
],["id","features"])

In [ ]:
df4.show()

In [ ]:
scaler = MinMaxScaler(inputCol="features",outputCol="sfeatures")

In [ ]:
model = scaler.fit(df4)

In [ ]:
df5 = model.transform(df4)

In [ ]:
df5.select("features","sfeatures").show()

## Standardizer

In [ ]:
from pyspark.ml.feature import StandardScaler

In [ ]:
scaler2 = StandardScaler(inputCol="features",outputCol="sfeatures",withStd=True,withMean=True)

model2 = scaler2.fit(df4)

df6 = model2.transform(df4)

In [ ]:
scaler2 = StandardScaler(inputCol="features",outputCol="sfeatures",withStd=True,withMean=True)
model2 = scaler2.fit(df4)
df6 = model2.transform(df4)

In [ ]:
df6.select("features","sfeatures").show()

## Ex: Normalizer

In [ ]:
cctv = spark.read.csv('./data/cctv.csv',header=True,inferSchema=True)

In [ ]:
cctv.show()

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
vectorAssembler = VectorAssembler(inputCols=["AT","V","AP","RH"],outputCol="features")

In [ ]:
cctv2 = vectorAssembler.transform(cctv)

In [ ]:
cctv2.show(20,False)

In [ ]:
scaler = MinMaxScaler(inputCol="features",outputCol="sfeatures")
model = scaler.fit(cctv2)
cctv3 = model.transform(cctv2)

In [ ]:
cctv3.select('sfeatures').show(20,False)

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

# Prepare training documents from a list of (id, text, label) tuples.
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

# Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# Fit the pipeline to training documents.
model = pipeline.fit(training)

# Prepare test documents, which are unlabeled (id, text) tuples.
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

# Make predictions on test documents and print columns of interest.
prediction = model.transform(test)
selected = prediction.select("id", "text", "probability", "prediction")
for row in selected.collect():
    rid, text, prob, prediction = row
    print("(%d, %s) --> prob=%s, prediction=%f" % (rid, text, str(prob), prediction))

## Tokenizer

In [ ]:
from pyspark.ml.feature import Tokenizer

df7 = spark.createDataFrame([
(1,"This is an introduction to Spark MLib"),
(2,"MLib contains libraries for classification and regression"),
(3,"It also contains supporting tools for pipelines")
],["id","sentence"])

df7.show()

token = Tokenizer(inputCol="sentence",outputCol="words")
df8 = token.transform(df7)

df8.show()

## TF-IDF

In [ ]:
from pyspark.ml.feature import HashingTF, IDF

hftoken = HashingTF(inputCol="words",outputCol="rawFeatures",numFeatures=20)
df9 = hftoken.transform(df8)

df9.take(1)

idf = IDF(inputCol="rawFeatures",outputCol="idf_features")
model3 = idf.fit(df9)
df10 = model3.transform(df9)

df10.take(1)